### Using `concurrent.futures` for Parallel Processing in Python

#### 1. Introduction to `concurrent.futures`
- `concurrent.futures` provides a high-level interface for asynchronously executing callables.
- Two main classes:
  - `ThreadPoolExecutor` for managing a pool of threads.
  - `ProcessPoolExecutor` for managing a pool of processes.

#### 2. Basic Example of Parallel Processing
- Using `ProcessPoolExecutor` to apply a function to each element in a list:

```python
import concurrent.futures

def process_element(element, multiplier):
    return element * multiplier

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
multiplier = 3

with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = [executor.submit(process_element, n, multiplier) for n in numbers]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

print(results)
```

#### 3. Handling Multiple Arguments
- To pass multiple arguments to the function, use `zip` and `itertools.repeat`:

```python
import concurrent.futures
import itertools

def process_element(element, multiplier):
    return element * multiplier

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
multiplier = 3

args = zip(numbers, itertools.repeat(multiplier))

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(lambda p: process_element(*p), args))

print(results)
```

#### 4. Debugging and Logging
- Adding logging to help debug and monitor the processing:

```python
import concurrent.futures
import logging
import itertools

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

def process_element(element, multiplier):
    logging.info(f'Processing element: {element}')
    return element * multiplier

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
multiplier = 3

args = zip(numbers, itertools.repeat(multiplier))

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(lambda p: process_element(*p), args))

print(results)
```

#### 5. Handling Exceptions
- Ensure that exceptions are handled properly within the function to prevent `BrokenProcessPool` errors:

```python
def process_element(element, multiplier):
    try:
        logging.info(f'Processing element: {element}')
        return element * multiplier
    except Exception as e:
        logging.error(f'Error processing element {element}: {e}')
        raise
```

#### 6. Limiting the Number of Workers
- Use the `max_workers` parameter to limit the number of threads or processes:

```python
with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
    results = list(executor.map(lambda p: process_element(*p), args))
```

#### 7. Comparison with `ThreadPoolExecutor`
- To compare or switch to `ThreadPoolExecutor`, the code structure remains largely the same:

```python
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    results = list(executor.map(lambda p: process_element(*p), args))
```

### Key Takeaways
- `concurrent.futures` is a powerful library for parallel processing using threads or processes.
- Proper handling of function arguments and logging can help debug and optimize parallel tasks.
- Exception handling within functions is crucial to prevent process pool errors.
- The `max_workers` parameter controls the level of parallelism.
- Testing with `ThreadPoolExecutor` can help identify if issues are specific to process-based parallelism.


In `concurrent.futures`, both `map` and `submit` are used to schedule tasks to be executed by a thread or process pool. However, they serve different purposes and have different usage patterns. Here’s a detailed comparison:

### `submit`
- **Usage**: Submits a single callable (function) for execution.
- **Returns**: A `Future` object representing the execution of the callable.
- **Advantages**:
  - Fine-grained control: Allows you to manage each task individually.
  - Can be used with any callable, not just functions with iterable inputs.
  - Supports different arguments for each callable.
  - Useful when you need to handle the results or exceptions of individual tasks.

**Example**:

```python
import concurrent.futures

def process_element(element, multiplier):
    return element * multiplier

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
multiplier = 3

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_element, n, multiplier) for n in numbers]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

print(results)
```

### `map`
- **Usage**: Submits a callable to be executed for each item in an iterable.
- **Returns**: An iterator equivalent to `map(func, *iterables)`, but the calls may be evaluated out-of-order.
- **Advantages**:
  - Simpler syntax for applying a function to an iterable.
  - Automatically collects results in the order they were submitted.
  - More concise when the function and arguments fit the pattern.

**Example**:

```python
import concurrent.futures

def process_element(element, multiplier):
    return element * multiplier

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
multiplier = 3

def process_element_wrapper(args):
    return process_element(*args)

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_element_wrapper, zip(numbers, [multiplier]*len(numbers))))

print(results)
```

### Key Differences
1. **Granularity**:
   - `submit` provides fine-grained control over each task.
   - `map` is higher-level and simpler when you need to apply a function to a sequence of inputs.

2. **Return Values**:
   - `submit` returns a `Future` object immediately.
   - `map` returns an iterator that yields results as tasks complete.

3. **Order**:
   - `submit` with `as_completed` can process results as they complete, potentially out of order.
   - `map` returns results in the order the tasks were submitted.

4. **Complexity**:
   - `submit` is more flexible and can handle varying arguments and more complex scenarios.
   - `map` is easier to use for straightforward applications of a function to a list of inputs.

### When to Use Each
- Use **`submit`** when:
  - You need fine-grained control over individual tasks.
  - Tasks are independent and have different arguments.
  - You need to handle exceptions or results individually.

- Use **`map`** when:
  - You have a single function to apply to an iterable.
  - You want results in the order they were submitted.
  - The syntax is simple and concise for your use case.

In [ ]:
#Multiprocess

import concurrent.futures as thread
import logging as logger
lst_to_process = [1,2,3,4,5,6,7,8]
multiplier = 5

def fun(x, multiplier):
    logger.info(f"proceesing element {x}")
    return x*multiplier

with thread.ThreadPoolExecutor(max_workers=2) as tp_executor:                      # Use max_workers to define parallel threads at a time
    futures = [tp_executor.submit(fun, n, multiplier) for n in lst_to_process]

    # result = [future.result() for future in thread.as_completed(futures)]        # When you don't care about order
    result = [future.result() for future in futures]                               # When you care about order

print(result)

2024-07-21 16:56:46,849 INFO:proceesing element 1
2024-07-21 16:56:46,855 INFO:proceesing element 2
2024-07-21 16:56:46,856 INFO:proceesing element 3
2024-07-21 16:56:46,862 INFO:proceesing element 4
2024-07-21 16:56:46,866 INFO:proceesing element 5
2024-07-21 16:56:46,868 INFO:proceesing element 6
2024-07-21 16:56:46,873 INFO:proceesing element 8
2024-07-21 16:56:46,871 INFO:proceesing element 7


[5, 10, 15, 20, 25, 30, 35, 40]


In [ ]:
#Multiprocess

import concurrent.futures as thread
import logging as logger
import itertools

lst_to_process = [1,2,3,4,5,6,7,8]
multiplier = 5

def fun(x, multiplier):
    logger.info(f"proceesing element {x}")
    return x*multiplier

list_of_args = zip(lst_to_process, itertools.repeat(multiplier))

with thread.ThreadPoolExecutor(max_workers=2) as tp_executor:                      # Use max_workers to define parallel threads at a time
    results = list(tp_executor.map(lambda arg : fun(*arg) , list_of_args))

print(results)

2024-07-21 17:21:19,243 INFO:proceesing element 1
2024-07-21 17:21:19,249 INFO:proceesing element 2
2024-07-21 17:21:19,256 INFO:proceesing element 4
2024-07-21 17:21:19,254 INFO:proceesing element 3
2024-07-21 17:21:19,259 INFO:proceesing element 5
2024-07-21 17:21:19,262 INFO:proceesing element 6
2024-07-21 17:21:19,264 INFO:proceesing element 7
2024-07-21 17:21:19,266 INFO:proceesing element 8


[5, 10, 15, 20, 25, 30, 35, 40]


In [2]:
# Tested on ide and it works
"""
import concurrent.futures

def process_element(element, multiplier):
    return element * multiplier

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
multiplier = 3

if __name__ == '__main__':

    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_element, n, multiplier) for n in numbers]
        results = [future.result() for future in concurrent.futures.as_completed(futures)]

    print(results)
"""

"\nimport concurrent.futures\n\ndef process_element(element, multiplier):\n    return element * multiplier\n\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nmultiplier = 3\n\nif __name__ == '__main__':\n\n    with concurrent.futures.ProcessPoolExecutor() as executor:\n        futures = [executor.submit(process_element, n, multiplier) for n in numbers]\n        results = [future.result() for future in concurrent.futures.as_completed(futures)]\n\n    print(results)\n"

The `itertools` module in Python is a powerful library that provides various functions to work with iterators. It includes functions for creating infinite iterators, terminating iterators, and combinatoric iterators. Here's a detailed explanation of the most commonly used functions in the `itertools` module, along with use cases and comments.

### Importing `itertools`
To use `itertools`, you need to import it:
```python
import itertools
```

### Infinite Iterators

1. **`count(start=0, step=1)`**
   - Returns an iterator that generates consecutive values starting from `start` and increments by `step`.

   ```python
   import itertools

   # Infinite counter starting from 10, incrementing by 2
   counter = itertools.count(10, 2)
   for i in range(5):
       print(next(counter))  # Output: 10, 12, 14, 16, 18
   ```

2. **`cycle(iterable)`**
   - Returns an iterator that cycles through the elements in `iterable` indefinitely.

   ```python
   # Cycles through the list indefinitely
   colors = itertools.cycle(['red', 'green', 'blue'])
   for i in range(6):
       print(next(colors))  # Output: red, green, blue, red, green, blue
   ```

3. **`repeat(object, times=None)`**
   - Returns an iterator that repeats the given object `times` times. If `times` is `None`, it repeats indefinitely.

   ```python
   # Repeats the string 'hello' 3 times
   repeated = itertools.repeat('hello', 3)
   for item in repeated:
       print(item)  # Output: hello, hello, hello
   ```

### Terminating Iterators

4. **`accumulate(iterable, func=operator.add)`**
   - Returns an iterator that yields accumulated sums, or results of a binary function (e.g., max).

   ```python
   import itertools
   import operator

   # Accumulate sums of the list
   data = [1, 2, 3, 4, 5]
   accumulated_sums = itertools.accumulate(data)
   print(list(accumulated_sums))  # Output: [1, 3, 6, 10, 15]

   # Accumulate maximum values
   accumulated_max = itertools.accumulate(data, func=operator.max)
   print(list(accumulated_max))  # Output: [1, 2, 3, 4, 5]
   ```

5. **`chain(*iterables)`**
   - Returns an iterator that yields elements from the first iterable until it is exhausted, then proceeds to the next iterable, until all of them are exhausted.

   ```python
   # Chain multiple lists into one iterator
   chained = itertools.chain([1, 2, 3], ['a', 'b', 'c'])
   print(list(chained))  # Output: [1, 2, 3, 'a', 'b', 'c']
   ```

6. **`compress(data, selectors)`**
   - Returns an iterator that filters elements from `data`, returning only those that have a corresponding element in `selectors` that evaluates to `True`.

   ```python
   # Compress elements where selectors are True
   data = ['a', 'b', 'c', 'd']
   selectors = [True, False, True, False]
   compressed = itertools.compress(data, selectors)
   print(list(compressed))  # Output: ['a', 'c']
   ```

7. **`dropwhile(predicate, iterable)`**
   - Drops elements from the iterable as long as the predicate is true; afterwards, returns every remaining element.

   ```python
   # Drop elements while the condition is true
   data = [1, 2, 3, 4, 5, 6]
   result = itertools.dropwhile(lambda x: x < 4, data)
   print(list(result))  # Output: [4, 5, 6]
   ```

8. **`takewhile(predicate, iterable)`**
   - Returns elements from the iterable as long as the predicate is true.

   ```python
   # Take elements while the condition is true
   data = [1, 2, 3, 4, 5, 6]
   result = itertools.takewhile(lambda x: x < 4, data)
   print(list(result))  # Output: [1, 2, 3]
   ```

9. **`filterfalse(predicate, iterable)`**
   - Returns elements of the iterable for which the predicate is false.

   ```python
   # Filter elements where the condition is false
   data = [1, 2, 3, 4, 5, 6]
   result = itertools.filterfalse(lambda x: x % 2 == 0, data)
   print(list(result))  # Output: [1, 3, 5]
   ```

10. **`groupby(iterable, key=None)`**
    - Returns consecutive keys and groups from the iterable. `key` is a function computing a key value for each element.

    ```python
    # Group elements by their value
    data = [1, 1, 2, 2, 2, 3, 3, 4]
    grouped = itertools.groupby(data)
    for key, group in grouped:
        print(key, list(group))  # Output: 1 [1, 1], 2 [2, 2, 2], 3 [3, 3], 4 [4]
    ```

11. **`islice(iterable, start, stop[, step])`**
    - Returns selected elements from the iterable, like slicing a list.

    ```python
    # Slice the list from index 1 to 4
    data = [0, 1, 2, 3, 4, 5]
    sliced = itertools.islice(data, 1, 5)
    print(list(sliced))  # Output: [1, 2, 3, 4]
    ```

12. **`starmap(func, iterable)`**
    - Returns an iterator whose values are returned from the function evaluated with an argument tuple taken from the given iterable.

    ```python
    import math

    # Apply the function to elements
    data = [(2, 5), (3, 2), (10, 3)]
    result = itertools.starmap(math.pow, data)
    print(list(result))  # Output: [32.0, 9.0, 1000.0]
    ```

### Combinatoric Iterators

13. **`product(*iterables, repeat=1)`**
    - Cartesian product of input iterables.

    ```python
    # Cartesian product of two lists
    result = itertools.product([1, 2], ['a', 'b'])
    print(list(result))  # Output: [(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]
    ```

14. **`permutations(iterable, r=None)`**
    - Returns successive r-length permutations of elements in the iterable.

    ```python
    # Permutations of length 2
    result = itertools.permutations([1, 2, 3], 2)
    print(list(result))  # Output: [(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]
    ```

15. **`combinations(iterable, r)`**
    - Returns successive r-length combinations of elements in the iterable.

    ```python
    # Combinations of length 2
    result = itertools.combinations([1, 2, 3], 2)
    print(list(result))  # Output: [(1, 2), (1, 3), (2, 3)]
    ```

16. **`combinations_with_replacement(iterable, r)`**
    - Returns successive r-length combinations of elements in the iterable, allowing individual elements to have successive repeats.

    ```python
    # Combinations with replacement of length 2
    result = itertools.combinations_with_replacement([1, 2, 3], 2)
    print(list(result))  # Output: [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)]
    ```

These are some of the most commonly used functions in the `itertools` module. Each function provides a unique way to handle iterators and can be very useful in various scenarios involving data manipulation and processing.

In [13]:
import itertools
# Test itertools islice
lst = [1,2,3,4,5]
print([x for x in itertools.islice(lst,1,4,2)])

[2, 4]


#functools module
The `functools` module in Python provides higher-order functions that act on or return other functions. This module contains various utilities that are useful for functional programming. Here is a detailed explanation of the `functools` module, including various use cases with comments.

### Importing `functools`
To use the `functools` module, you need to import it:
```python
import functools
```

### `functools.partial`
The `partial` function allows you to fix a certain number of arguments of a function and generate a new function.

```python
import functools

def multiply(a, b):
    return a * b

# Create a new function that multiplies any number by 2
double = functools.partial(multiply, 2)

print(double(5))  # Output: 10
```

### `functools.update_wrapper` and `functools.wraps`
These functions are used to update the wrapper function to look more like the wrapped function by copying attributes such as the module, name, and docstring. This is useful when creating decorators.

```python
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Before function call")
        result = func(*args, **kwargs)
        print("After function call")
        return result
    return wrapper

@my_decorator
def say_hello():
    """A function that says hello"""
    print("Hello!")

say_hello()
print(say_hello.__name__)  # Output: say_hello
print(say_hello.__doc__)   # Output: A function that says hello
```

### `functools.lru_cache`
The `lru_cache` decorator allows you to cache the results of a function, using a Least Recently Used (LRU) cache to store the results. This is useful for optimizing expensive function calls.

```python
import functools

@functools.lru_cache(maxsize=4)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))  # Output: 55
print(fibonacci.cache_info())  # Cache info
```

### `functools.reduce`
The `reduce` function applies a binary function cumulatively to the items of an iterable, from left to right, to reduce the iterable to a single value.

```python
import functools

numbers = [1, 2, 3, 4, 5]
result = functools.reduce(lambda x, y: x * y, numbers)
print(result)  # Output: 120
```


### `functools.partialmethod`
The `partialmethod` function is similar to `partial` but is used to define a new method with fixed arguments.

```python
import functools

class MyClass:
    def __init__(self, name):
        self.name = name

    def greet(self, message):
        print(f"{self.name} says {message}")

    greet_hello = functools.partialmethod(greet, "Hello!")

obj = MyClass("Alice")
obj.greet_hello()  # Output: Alice says Hello!
```

### `functools.cmp_to_key`
The `cmp_to_key` function converts an old-style comparison function to a key function. This is useful for sorting.

```python
import functools

def compare(a, b):
    return (a > b) - (a < b)

numbers = [5, 2, 9, 1]
sorted_numbers = sorted(numbers, key=functools.cmp_to_key(compare))
print(sorted_numbers)  # Output: [1, 2, 5, 9]
```

### Summary

The `functools` module in Python provides various utilities for higher-order functions and functional programming. It includes tools for creating partial functions, updating wrapper functions, caching results, reducing iterables, ensuring total ordering, creating single-dispatch generic functions, defining partial methods, and converting comparison functions. By understanding and utilizing these tools, you can write more efficient, readable, and maintainable code.

In [27]:
# Implementing decorators
import functools                                  # fixed
def modify_func(func):                            # fixed
    @functools.wraps(func)                            # fixed
    def wrapper(*args, **kwargs):                            
        print("preprocess")
        result = func(*args, **kwargs)                            # fixed
        print("postprocess")
        return result                            # fixed
    return wrapper                            # fixed

In [28]:
@modify_func
def hello(var):
    print(f"Hi {var}")

In [29]:
hello("Rupesh")

preprocess
Hi Rupesh
postprocess


In [51]:
# functools reduce function

import functools

numbers = [1, 2, 3, 4, 5]
result = functools.reduce(lambda x, y: x * y, numbers)
print(result)  # Output: 120

120


In [62]:
# return list of even numbers
x = [1,2,3]
fun = lambda x : x%2 == 0 
print(list(filter(fun, x)))


print(f"sum(x) {sum(x)}")

print(f"min(x) {min(x)}")

[2]
sum(x) 6
min(x) 1
